In [130]:
import pandas as pd
import geopandas as gpd
from shapely import Point, LineString, MultiLineString
from geopy.geocoders import Nominatim
import requests
import networkx as nx
from shapely.wkt import loads
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import to_hex
import dash_bootstrap_components as dbc
from dash_extensions.enrich import DashProxy, MultiplexerTransform
import dash_leaflet as dl
from dash import html, dcc
import plotly.express as px
from dash.dependencies import Input, Output, State
import webbrowser
import random
import pickle
import osmnx as ox
import warnings
warnings.filterwarnings("ignore")

In [132]:
def geolocate(address1=None, address2=None):
    geolocator = Nominatim(user_agent="xxx")
    if address2 and address1:
        address = f'{address1}, {address2}'
    elif address1:
        address = address1
    elif address2:
        address = address2
    else:
        print('No address provided')
        return None
    try:
        address = geolocator.geocode(address, timeout=10)
        if address:
            coords = [address.latitude,address.longitude]
            return coords
        else:
            try:
                address = geolocator.geocode(address1, timeout=10)
                if address:
                    coords = [address.latitude,address.longitude]
                    return coords
                else:
                    print('error:',address)
                    return []
            except:
                print('error:',address)
                return []
    except:
        print('error:',address)
        return []
    
def find_path(G, source, target, method='min_connections'):
    if method == 'min_connections':
        try:
            return nx.shortest_path(G, source, target)
        except:
            return []
#             print('error:',source,target)
    elif method == 'min_length':
        try:
            return nx.dijkstra_path(G, source, target, weight='Distance')
        except:
            return []
#             print('error:',source,target)
    else:
        print('Invalid Method')
        
def shortest_path_edges(path):

    edges = []
    try:
        for node in range(len(path)-1):
            O = path[node]
            D = path[node+1]
            edges.append([O,D])
    except:
        edges.append([])
    return edges

def normalize_column(column, ascending=True):
    norm =  (column - column.min()) / (column.max() - column.min())
    if ascending:
        return norm
    else:
        return 1-norm

In [131]:
airports = gpd.read_file('../Resources/OpenFlights/airports')

# airports = airports[airports['AirportNam'].str.find('International') != -1]

airports.Country = airports.Country.replace(
    {
        "Congo (Brazzaville)":"Congo",
        "Congo (Kinshasa)":"Democratic Republic of the Congo",
        "Cote d'Ivoire":"Côte d'Ivoire",
        "Czech Republic":"Czechia",
        "East Timor":"Timor-Leste",
        "Swaziland":"Eswatini",
        "United States":"United States of America",
        "West Bank":"Palestine",
        "Burma":"Myanmar"
    }
)

airports['AirportGeometry'] = airports['geometry']

airports = airports.rename(columns={'City':'AirportCity', 'Country':'AirportCountry'})

airport_columns = ['AirportID','AirportGeometry','AirportNam','AirportCity','AirportCountry']

airports = airports[airport_columns].set_geometry('AirportGeometry')

routes = gpd.read_file('../Resources/OpenFlights/routes')
routes['Count'] = 1

routes_grouped = routes.groupby(['SAirportID','DAirportID','SourceCity','DestCity']).agg({'geometry':'first','Count':'count','Distance':'mean'}).reset_index().set_geometry('geometry')
routes_grouped = routes_grouped[routes_grouped['Distance']!=0]

# G = nx.DiGraph()
# G.add_nodes_from(list(airports['AirportID'].unique()))
# G.add_edges_from([list(pair) for pair in list(routes_grouped[['SAirportID','DAirportID']].values)])

G = nx.from_pandas_edgelist(
    routes_grouped,
    source='SAirportID',
    target='DAirportID',
    edge_attr='Distance',
    create_using=nx.DiGraph()
)

G.remove_nodes_from(list(nx.isolates(G)))
# print(len(G.nodes))
airports = airports[airports['AirportID'].isin(list(G.nodes))]

In [133]:
df = pd.read_excel('OtherEvents.xlsx')

In [134]:
df['OriginCoords'] = df.apply(lambda row: geolocate(f"{row['OriginCity']}, {row['OriginCountry']}"), axis=1)

In [135]:
df['DestinationCoords'] = df.apply(lambda row: geolocate(f"{row['DestinationCity']}, {row['DestinationCountry']}"), axis=1)

In [137]:
df['OriginGeometry'] = gpd.GeoSeries(gpd.points_from_xy(
    df['OriginCoords'].apply(lambda c: c[1]),
    df['OriginCoords'].apply(lambda c: c[0]),
    crs=4326
))

df['DestinationGeometry'] = gpd.GeoSeries(gpd.points_from_xy(
    df['DestinationCoords'].apply(lambda c: c[1]),
    df['DestinationCoords'].apply(lambda c: c[0]),
    crs=4326
))

df = gpd.GeoDataFrame(df)
df2 = df.set_geometry('OriginGeometry').sjoin_nearest(airports).rename(
    columns = {col:f'Origin{col}' for col in airport_columns}
).drop('index_right',axis=1).set_geometry('DestinationGeometry').sjoin_nearest(airports).rename(
    columns = {col:f'Destination{col}' for col in airport_columns}
).drop('index_right',axis=1)

In [202]:
df2['shortest_path'] = df2.apply(
    lambda row: find_path(G, row['OriginAirportID'],row['DestinationAirportID']), axis=1
)

df2['shortest_path_edges'] = df2['shortest_path'].apply(
    shortest_path_edges
).apply(lambda x: [[x.index(c),c] for c in x])

In [118]:
PRIVATE_JET_OCUP = 50 #Estimated based on 
PRIVATE_JET_EF = 3.044726409584053 # 4.9 kg per mile / 1.60934 km/mile(https://flybitlux.com/what-is-the-carbon-footprint-of-a-private-jet/#:~:text=A%20typical%20private%20jet%20emits,grams%20per%20passenger%20per%20kilometer.)

In [264]:
df_flights = df2[df2['Event']=='F1 2024'].explode('shortest_path_edges')

df_flights['path_index'] = df_flights['shortest_path_edges'].apply(lambda x: x[0] if type(x)==list else np.nan)
df_flights['SAirportID'] = df_flights['shortest_path_edges'].apply(lambda x: x[1][0] if type(x)==list else np.nan)
df_flights['DAirportID'] = df_flights['shortest_path_edges'].apply(lambda x: x[1][1] if type(x)==list else np.nan)

df_flights = gpd.GeoDataFrame(df_flights.merge(routes_grouped[['SAirportID', 'DAirportID', 'Distance','geometry']]), crs=4326)
df_flights['coords'] = df_flights.geometry.apply(lambda x: list(x.coords)).apply(lambda x: [[x.index(c),c] for c in x])
df_flights['coords_edges'] = df_flights['coords'].apply(
    shortest_path_edges
).apply(lambda x: [[x.index(c),c] for c in x])
df_flights = df_flights.explode('coords_edges')
df_flights

centroid = df_flights.total_bounds
centroid_x = (centroid[0] + centroid[2]) / 2
centroid_y = (centroid[1] + centroid[3]) / 2
centroid = [centroid_y,centroid_x]

df_flights['pathlength'] = df_flights.length
df_flights['leglength'] = (df_flights.length/df_flights['pathlength'])*df_flights['Distance']
df_flights['AcumLength'] = (df_flights.sort_values(['Event','Order','path_index'])['leglength'].cumsum()/1000).astype(int)


df_flights['FlightCO2Em'] = ((df_flights['Distance']/1000) * PRIVATE_JET_EF)/1000

In [265]:
df_flights2 = df2[df2['Event']=='Taylor Swift Tour'].explode('shortest_path_edges')

df_flights2['path_index'] = df_flights2['shortest_path_edges'].apply(lambda x: x[0] if type(x)==list else np.nan)
df_flights2['SAirportID'] = df_flights2['shortest_path_edges'].apply(lambda x: x[1][0] if type(x)==list else np.nan)
df_flights2['DAirportID'] = df_flights2['shortest_path_edges'].apply(lambda x: x[1][1] if type(x)==list else np.nan)

df_flights2 = gpd.GeoDataFrame(df_flights2.merge(routes_grouped[['SAirportID', 'DAirportID', 'Distance','geometry']]), crs=4326)
df_flights2['coords'] = df_flights2.geometry.apply(lambda x: list(x.coords)).apply(lambda x: [[x.index(c),c] for c in x])
df_flights2['coords_edges'] = df_flights2['coords'].apply(
    shortest_path_edges
).apply(lambda x: [[x.index(c),c] for c in x])
df_flights2 = df_flights2.explode('coords_edges')
df_flights2

centroid2 = df_flights2.total_bounds
centroid_x = (centroid2[0] + centroid2[2]) / 2
centroid_y = (centroid2[1] + centroid2[3]) / 2
centroid2 = [centroid_y,centroid_x]

df_flights2['pathlength'] = df_flights2.length
df_flights2['leglength'] = (df_flights2.length/df_flights2['pathlength'])*df_flights2['Distance']
df_flights2['AcumLength'] = (df_flights2.drop_duplicates(['Event','Order','path_index']).sort_values(['Event','Order','path_index'])['leglength'].cumsum()/1000).astype(int)


df_flights2['FlightCO2Em'] = ((df_flights2['Distance']/1000) * PRIVATE_JET_EF)/1000

In [266]:
app = DashProxy(
    transforms=[MultiplexerTransform()], 
    external_stylesheets=[dbc.themes.DARKLY],
    external_scripts=[
        {'src': 'https://cdnjs.cloudflare.com/ajax/libs/html2canvas/1.4.0/html2canvas.min.js'}
    ]
)

app.title= 'F1 Tracker ✈️'

SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "20rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
    "fontSize":"80%",
    "lineheight":"80%",
    "overflow": "scroll"
}

CONTENT_STYLE = {
    "margin-left": "20rem"
}

colorscale = ['#0b090a', '#161a1d', '#660708', '#a4161a', '#e5383b', '#b1a7a6', '#d3d3d3', '#f5f3f4']

basemap = [
    dl.TileLayer(
        url='https://tiles.stadiamaps.com/tiles/alidade_smooth_dark/{z}/{x}/{y}{r}.png',
        attribution='&copy; <a href="https://www.stadiamaps.com/" target="_blank">Stadia Maps</a> &copy; <a href="https://openmaptiles.org/" target="_blank">OpenMapTiles</a> &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',

    #     url = 'https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}',
    #     attribution = 'Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ',
        maxZoom=16
    ),
]

#PREPARE GLOBAL MAP
temp_map = basemap.copy()

df_flights[(df_flights['AcumLength']<=-1)].apply(
        lambda row: temp_map.append(
            dl.Polyline(
                positions=[(coord[1], coord[0]) for coord in row['geometry'].coords],
                color='#D52221',
                weight=2
            )
        ),axis=1
)


app.layout = dbc.Container([
    dbc.Row(
        [
            dbc.Row(
                dbc.Label(' F1 Tracker ✈️', style={'color':'white'}),
                style={'backgroundColor':colorscale[0],'fontSize':'100%','fontWeight':'bold','padding':'0px 0px 0px 10px'}
            ),
            dbc.Row([
                    dbc.Col([
                        dbc.Label('Flight Distance Travelled', style={'color':'white','fontSize':'70%','fontWeight':'bold','padding':'0px 0px 0px 10px'}),
                        dcc.Slider(
                            0, df_flights['AcumLength'].max(),
                            value=0,id='slider-prior',
                            tooltip={"placement": "bottom", "always_visible": False}
                        ),
                    ], style={'backgroundColor':colorscale[1]},width=5),
                    dbc.Col([
                        dbc.Label(
                            "{:,.0f}".format(df_flights[(df_flights['AcumLength']<=-1)]['FlightCO2Em'].sum()),
                            id='emissions-tracker',
                            style={'color':'white','fontSize':'300%','fontWeight':'bold','padding':'0px 0px 0px 10px','text-align':'right'}
                        ),
                    ], style={'backgroundColor':colorscale[1],'text-align':'right'},width=4),
                dbc.Col([
                    dbc.Row(dbc.Label('ton CO2',style={'color':'white','fontSize':'200%','fontWeight':'bold','padding':'0px 0px 0px 10px'})),
                    dbc.Row(dbc.Label('generated',style={'color':'white','fontSize':'100%','fontWeight':'bold','padding':'0px 0px 0px 10px','height':'20px'})),
                ],style={'backgroundColor':colorscale[1]}
                )
                ],style={'padding':'0px 0px 0px 0px'}
            ),
        ],
        className='sticky-top fluid',style={'padding':'0px 0px 0px 10px','backgroundColor':colorscale[0]}
    ),
    dbc.Row([
        dbc.Col(dl.MapContainer(
            id='map-historic',
            children=temp_map,
            style={
                'width': '100%', 
                'height': '600px'
            },
            center=centroid,
            zoom=2
        ),width=12)
    ],className="g-0",style={'backgroundColor':'#212222'}),
],fluid=True,style={'fontFamily':'Montserrat','backgroundColor':'#212222'})

@app.callback(
    [
        Output('map-historic','children'),
        Output('emissions-tracker','children')
    ],
    [
        Input('slider-prior','value')
    ]
)

def update_prioritization(vis_prior):
    
    temp_map = basemap.copy()
    
    df_flights[(df_flights['AcumLength']<=vis_prior)].apply(
            lambda row: temp_map.append(
                dl.Polyline(
                    positions=[(coord[1], coord[0]) for coord in row['geometry'].coords],
                    color='#D52221',
                    weight=2
                )
            ),axis=1
    )
    return temp_map,  "{:,.0f}".format(df_flights[(df_flights['AcumLength']<=vis_prior)]['FlightCO2Em'].sum())

server = app.server
if __name__ == "__main__":
    webbrowser.open('http://localhost:8050')
    app.run_server(debug=False, port=8050)

In [255]:
app = DashProxy(
    transforms=[MultiplexerTransform()], 
    external_stylesheets=[dbc.themes.DARKLY],
    external_scripts=[
        {'src': 'https://cdnjs.cloudflare.com/ajax/libs/html2canvas/1.4.0/html2canvas.min.js'}
    ]
)

app.title= 'F1 Tracker ✈️'

SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "20rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
    "fontSize":"80%",
    "lineheight":"80%",
    "overflow": "scroll"
}

CONTENT_STYLE = {
    "margin-left": "20rem"
}

colorscale = ['#0b090a', '#161a1d', '#660708', '#a4161a', '#e5383b', '#b1a7a6', '#d3d3d3', '#f5f3f4']

basemap = [
    dl.TileLayer(
        url='https://tiles.stadiamaps.com/tiles/alidade_smooth_dark/{z}/{x}/{y}{r}.png',
        attribution='&copy; <a href="https://www.stadiamaps.com/" target="_blank">Stadia Maps</a> &copy; <a href="https://openmaptiles.org/" target="_blank">OpenMapTiles</a> &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',

    #     url = 'https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}',
    #     attribution = 'Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ',
        maxZoom=16
    ),
]

#PREPARE GLOBAL MAP
temp_map = basemap.copy()

df_flights[(df_flights['AcumLength']<=-1)].apply(
        lambda row: temp_map.append(
            dl.Polyline(
                positions=[(coord[1], coord[0]) for coord in row['geometry'].coords],
                color='#D52221',
                weight=2
            )
        ),axis=1
)


app.layout = dbc.Container([
    dbc.Row(
        [
            dbc.Row(
                dbc.Label(' F1 Tracker ✈️', style={'color':'white'}),
                style={'backgroundColor':colorscale[0],'fontSize':'100%','fontWeight':'bold','padding':'0px 0px 0px 10px'}
            ),
            dbc.Row([
                    dbc.Col([
                        dbc.Label('Flight Distance Travelled', style={'color':'white','fontSize':'70%','fontWeight':'bold','padding':'0px 0px 0px 10px'}),
                        dcc.Slider(
                            0, df_flights['AcumLength'].max(),
                            value=0,id='slider-prior',
                            tooltip={"placement": "bottom", "always_visible": False}
                        ),
                    ], style={'backgroundColor':colorscale[1]},width=5),
                    dbc.Col([
                        dbc.Label(
                            "{:,.0f}".format(df_flights[(df_flights['AcumLength']<=-1)]['FlightCO2Em'].sum()),
                            id='emissions-tracker',
                            style={'color':'white','fontSize':'300%','fontWeight':'bold','padding':'0px 0px 0px 10px','text-align':'right'}
                        ),
                    ], style={'backgroundColor':colorscale[1],'text-align':'right'},width=4),
                dbc.Col([
                    dbc.Row(dbc.Label('ton CO2',style={'color':'white','fontSize':'200%','fontWeight':'bold','padding':'0px 0px 0px 10px'})),
                    dbc.Row(dbc.Label('generated',style={'color':'white','fontSize':'100%','fontWeight':'bold','padding':'0px 0px 0px 10px','height':'20px'})),
                ],style={'backgroundColor':colorscale[1]}
                )
                ],style={'padding':'0px 0px 0px 0px'}
            ),
        ],
        className='sticky-top fluid',style={'padding':'0px 0px 0px 10px','backgroundColor':colorscale[0]}
    ),
    dbc.Row([
        dbc.Col(dl.MapContainer(
            id='map-historic',
            children=temp_map,
            style={
                'width': '100%', 
                'height': '600px'
            },
            center=centroid,
            zoom=2
        ),width=12)
    ],className="g-0",style={'backgroundColor':'#212222'}),
],fluid=True,style={'fontFamily':'Montserrat','backgroundColor':'#212222'})

@app.callback(
    [
        Output('map-historic','children'),
        Output('emissions-tracker','children')
    ],
    [
        Input('slider-prior','value')
    ]
)

def update_prioritization(vis_prior):
    
    temp_map = basemap.copy()
    
    df_flights[(df_flights['AcumLength']<=vis_prior)].apply(
            lambda row: temp_map.append(
                dl.Polyline(
                    positions=[(coord[1], coord[0]) for coord in row['geometry'].coords],
                    color='#D52221',
                    weight=2
                )
            ),axis=1
    )
    return temp_map,  "{:,.0f}".format(df_flights[(df_flights['AcumLength']<=vis_prior)]['FlightCO2Em'].sum())

server = app.server
if __name__ == "__main__":
    webbrowser.open('http://localhost:8050')
    app.run_server(debug=False, port=8050)

In [267]:
app2 = DashProxy(
    transforms=[MultiplexerTransform()], 
    external_stylesheets=[dbc.themes.DARKLY],
    external_scripts=[
        {'src': 'https://cdnjs.cloudflare.com/ajax/libs/html2canvas/1.4.0/html2canvas.min.js'}
    ]
)

app2.title= 'Taylor Tracker ✈️'

SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "20rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
    "fontSize":"80%",
    "lineheight":"80%",
    "overflow": "scroll"
}

CONTENT_STYLE = {
    "margin-left": "20rem"
}

colorscale = ['#0b090a', '#161a1d', '#660708', '#a4161a', '#e5383b', '#b1a7a6', '#d3d3d3', '#f5f3f4']

basemap = [
    dl.TileLayer(
        url='https://tiles.stadiamaps.com/tiles/alidade_smooth_dark/{z}/{x}/{y}{r}.png',
        attribution='&copy; <a href="https://www.stadiamaps.com/" target="_blank">Stadia Maps</a> &copy; <a href="https://openmaptiles.org/" target="_blank">OpenMapTiles</a> &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',

    #     url = 'https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}',
    #     attribution = 'Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ',
        maxZoom=16
    ),
]

#PREPARE GLOBAL MAP
temp_map2 = basemap.copy()

df_flights2[(df_flights2['AcumLength']<=-1)].apply(
        lambda row: temp_map2.append(
            dl.Polyline(
                positions=[(coord[1], coord[0]) for coord in row['geometry'].coords],
                color='#D52221',
                weight=2
            )
        ),axis=1
)


app2.layout = dbc.Container([
    dbc.Row(
        [
            dbc.Row(
                dbc.Label(' Taylor Tracker ✈️', style={'color':'white'}),
                style={'backgroundColor':colorscale[0],'fontSize':'100%','fontWeight':'bold','padding':'0px 0px 0px 10px'}
            ),
            dbc.Row([
                    dbc.Col([
                        dbc.Label('Flight Distance Travelled', style={'color':'white','fontSize':'70%','fontWeight':'bold','padding':'0px 0px 0px 10px'}),
                        dcc.Slider(
                            0, df_flights2['AcumLength'].max(),
                            value=0,id='slider-prior',
                            tooltip={"placement": "bottom", "always_visible": False}
                        ),
                    ], style={'backgroundColor':colorscale[1]},width=5),
                    dbc.Col([
                        dbc.Label(
                            "{:,.0f}".format(df_flights2[(df_flights2['AcumLength']<=-1)]['FlightCO2Em'].sum()),
                            id='emissions-tracker',
                            style={'color':'white','fontSize':'300%','fontWeight':'bold','padding':'0px 0px 0px 10px','text-align':'right'}
                        ),
                    ], style={'backgroundColor':colorscale[1],'text-align':'right'},width=4),
                dbc.Col([
                    dbc.Row(dbc.Label('ton CO2',style={'color':'white','fontSize':'200%','fontWeight':'bold','padding':'0px 0px 0px 10px'})),
                    dbc.Row(dbc.Label('generated',style={'color':'white','fontSize':'100%','fontWeight':'bold','padding':'0px 0px 0px 10px','height':'20px'})),
                ],style={'backgroundColor':colorscale[1]}
                )
                ],style={'padding':'0px 0px 0px 0px'}
            ),
        ],
        className='sticky-top fluid',style={'padding':'0px 0px 0px 10px','backgroundColor':colorscale[0]}
    ),
    dbc.Row([
        dbc.Col(dl.MapContainer(
            id='map-historic',
            children=temp_map2,
            style={
                'width': '100%', 
                'height': '600px'
            },
            center=centroid,
            zoom=2
        ),width=12)
    ],className="g-0",style={'backgroundColor':'#212222'}),
],fluid=True,style={'fontFamily':'Montserrat','backgroundColor':'#212222'})

@app2.callback(
    [
        Output('map-historic','children'),
        Output('emissions-tracker','children')
    ],
    [
        Input('slider-prior','value')
    ]
)

def update_prioritization(vis_prior):
    
    temp_map2 = basemap.copy()
    
    df_flights2[(df_flights2['AcumLength']<=vis_prior)].apply(
            lambda row: temp_map2.append(
                dl.Polyline(
                    positions=[(coord[1], coord[0]) for coord in row['geometry'].coords],
                    color='#D52221',
                    weight=2
                )
            ),axis=1
    )
    return temp_map2,  "{:,.0f}".format(df_flights2[(df_flights2['AcumLength']<=vis_prior)]['FlightCO2Em'].sum())

server2 = app2.server
if __name__ == "__main__":
    webbrowser.open('http://localhost:8020')
    app2.run_server(debug=False, port=8020)